In [236]:
from TimeTagger import FileReader
import numpy as np
import easygui
import numpy as np
import matplotlib.pyplot as plt

We know that H and V got swapped in BOB so I must correct for it in code. 

In [237]:
#GLOBAL DEFINITIONS
H = 1
V = 2
A = 3
D = 4


In [238]:
#folder = r'C:\Data\\'
folder = "./swabian/4thNov2020_WithBeacon"
#file = '2.97mW_1.1.ttbin'
file = "3_5mW_00dB_witB.1.ttbin"
filepath = folder + '/' + file

#filepath = easygui.fileopenbox(default = folder)

In [239]:
def loadSwabianTimeStamps(fileIn, t = np.inf):
    
        file_reader = FileReader(fileIn)
        stamps = []
        channel = []
        
        
        while file_reader.hasData():
            n_events = 100000; # Number of events to read at once
            #n_events = 200000; # Number of events to read at once
            
            # Read at most n_events.
            data = file_reader.getData(n_events);  # data is an instance of TimeTagStreamBuffer

            channelTmp = data.getChannels()
            stampsTmp = data.getTimestamps()
          #  overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
          #  missed_events = data.getMissedEvents()
            
            for i in range(len(channelTmp)):
                channel.append(channelTmp[i])
                stamps.append(stampsTmp[i])
            break
            if stamps[-1] - stamps[0] > t:
                break
        return stamps, channel


In [240]:
stamps, channel = loadSwabianTimeStamps(filepath)

In [241]:
channel[:20]

[7, 4, 1, 7, 5, 8, 2, 5, 2, 6, 7, 5, 4, 8, 7, 8, 4, 3, 7, 6]

In [242]:
#seperate out alice's and bob's events
astamps = []
achannel = []
bstamps = []
bchannel = []
for (s,c) in zip(stamps, channel):
    if (c <=4):
        astamps.append(s)
        achannel.append(c)
    else:
        bstamps.append(s)
        nc = c-4
        
        #23 Oct Data
        if (nc == H):
            nc = A
        elif (nc == V):
            nc = D
        elif (nc == A):
            nc = H
        elif (nc == D):
            nc = V
        else :
            print ("Error! unknown channel")
            break

        '''    
        #1st Oct data
        
        if (nc == H):
            nc = D
        elif (nc == V):
            nc = A
        elif (nc == A):
            nc = H
        elif (nc == D):
            nc = V
        else :
            print ("Error! unknown channel")
            break
        
        '''
        '''if (nc == V):
            nc = D
        elif (nc == D):
            nc = V
        
        if (nc == A):
            nc = H
        elif (nc == H):
            nc = A
            
        if (nc == A):
            nc = D
        elif (nc ==D):
            nc = A
        '''
        '''if (nc == A):
            nc = H
        elif (nc == H):
            nc = A
        elif (nc == V):
            nc = D
        elif (nc == D):
            nc = V
        '''
        
        '''#24 Aug Data
        if ( nc == H ):
            nc = A
        elif ( nc == V):
            nc = D
        elif ( nc == A):
            nc = V
        elif (nc == D):
            nc = H
        else :
            print ("Error! unknown channel")
            break
        '''    
        '''if ( nc == H):
            nc = D
        elif (nc==D):
            nc = H
        
        if ( nc == A):
            nc = V
        elif (nc ==V):
            nc = A
        
        if ( nc == A):
            nc = D
        elif ( nc == D):
            nc = A'''
        
        '''#28th and 29th July data
        if (nc == D):
            nc = H
        elif (nc == H):
            nc = A
        elif (nc == A):
            nc = V
        elif (nc== V):
            nc = D'''
        
        '''
        # 14 july data
        if (nc==3):
            nc = 4
        elif (nc == 4):
            nc = 3
            
        '''        
        bchannel.append(nc)

In [243]:
len(astamps)

46453

In [244]:
len(bstamps)

53547

In [245]:
577447917

577447917

In [246]:
len(bstamps)+len(astamps)

100000

In [247]:
#convert the resolution to 1/8 of a nanosecond
#saverawevents

In [248]:
100*(astamps[-1] - astamps[0])/10**12

1.1087489914

In [249]:
achannel[-1]

3

In [250]:
astamps[:10]

[1251283797374,
 1251284105013,
 1251284573523,
 1251284736246,
 1251284973058,
 1251285306850,
 1251285310103,
 1251286031053,
 1251286041214,
 1251286258970]

In [251]:
achannel[:10]

[4, 1, 2, 2, 4, 4, 3, 3, 4, 2]

In [252]:
len(astamps)

46453

In [253]:
2197945792/64

34342903.0

In [263]:
def correct_timestamp(stamps,detectors,step_size_ps=250,sH=0,sV=0,sA=0,sD=0):
    cstamp=[]
    ts = 0
    for (s,d) in zip (stamps,detectors):
        ts = s
        if(d==H): 
            ts+=sH*step_size_ps
        elif(d==V):
            ts+=sV*step_size_ps
        elif(d==A):
            ts+=sA*step_size_ps
        elif(d==D):
            ts+=sD*step_size_ps      
        cstamp.append(ts)
        
    print (step_size_ps*sH,step_size_ps*sV,step_size_ps*sA,step_size_ps*sD)
    
    return cstamp

In [264]:
#correct alice HV delay missmatch add 500ps to alice's V 
#astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=-1*500,sA=0,sD=-2*500) #28 july data
#astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=-2*500,sA=0,sD=-2*500) # 24 Aug data
#astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=-2*500,sA=-3*500,sD=-1*500) # 23rd Oct data
astamps_corrected = correct_timestamp(astamps,achannel,step_size_ps=250,sH=0,sV=-4,sA=-1,sD=-4) # 26rd Oct data


0 -1000 -250 -1000


In [265]:
## h = g2_heisto(astamps_corrected, achannel,bstamps, bchannel, amd=H, bmd=H,cwindow=500,search_steps=30,search_direction=-1)
#print (h)
#h.index(max(h))

In [266]:
### correct Bob
# add 7 To Bob's A basis
#bstamps_corrected = correct_timestamp(bstamps,bchannel,sH=0,sV=1*500,sA=-5*500,sD=1*500) # 28 july data
#bstamps_corrected = correct_timestamp(bstamps,bchannel,sH=0,sV=1*500,sA=1*500,sD=1*500) # 24 Aug data
#bstamps_corrected = correct_timestamp(bstamps,bchannel,sH=0,sV=-1*500,sA=0,sD=0) # 23rd Oct data
bstamps_corrected = correct_timestamp(bstamps,bchannel,step_size_ps = 250,sH=0,sV=0,sA=1,sD=1) # 26rd Oct data


0 0 250 250


In [268]:
hh = {}

In [383]:
def find_beacons(stamps,channel,cwindow,d_threshold ):
    i = 0
    beacon_indexes = []
    while(i<len(stamps)-3): 
        found_bitmap = np.zeros((5,), dtype=int)
        d_count = 0
        shift_i = 0
        fd = channel[i]
        ft = stamps[i]
        found_bitmap[fd] = 1
        d_count +=1
        
        for j in range(1,4):
            cd = channel[i+j]
            ct = stamps[i+j]
            shift_i = j
            
            if abs(ct-ft) > cwindow :
                break
                
            if found_bitmap[cd]==1:
                break
            else:                
                found_bitmap[cd] = 1
                d_count +=1
        
        if d_count >= d_threshold :
            beacon_indexes.append(i)
            
            #ignore next 200 nanosecond i.e 200,000 pico seconds
            while ((i+shift_i) < (len(stamps)-3))  and ((stamps[i+shift_i] - ft) < 200000) :
                
                shift_i+=1
            
        i = i+shift_i
    
    return beacon_indexes
            
        
        
        

In [424]:
alice_beacon_indexes = find_beacons(astamps_corrected,achannel,cwindow=2000,d_threshold=3)
len(alice_beacon_indexes)


110

In [425]:
200*10**-9*10000

0.002

In [426]:
i =0
a = alice_beacon_indexes[i]
b = alice_beacon_indexes[i+1]

interval = (astamps_corrected[b]-astamps_corrected[a])/10**12
frequency = 1/interval
print (frequency, interval*10**12)

10000.004400001935 99999956.0


In [427]:
missing_count = 0
for i in range (len(alice_beacon_indexes)-1):
    a = alice_beacon_indexes[i]
    b = alice_beacon_indexes[i+1]
    interval = (astamps_corrected[b]-astamps_corrected[a])/10**12
    frequency = 1/interval

    if ( abs(frequency - 10000)>1) :
        print ("error at: ", i)
        missing_count +=1

print(missing_count)

error at:  34
1


In [416]:
((astamps_corrected[-1]-astamps_corrected[0])/10**12) *10000

110.87490663999999